<a href="https://colab.research.google.com/github/oksuzian/Mu2eANL/blob/master/Macros/AgingCountersTestStandUVa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Init and load data

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy.polynomial.polynomial as poly
import numpy as np
import scipy
from scipy.optimize import curve_fit
import statistics
import math
from datetime import datetime, timedelta

In [9]:
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    main_path = '/content/drive/MyDrive/'
    print(main_path)
except:
    main_path = ''
main_path = '/content/drive/MyDrive/'

Mounted at /content/drive
/content/drive/MyDrive/


In [10]:
!tail drive/MyDrive/ScanningData_NaturalAging.txt

	2/6/2021 5:46 PM	0	0.0856	-0.0004	0.3043	0.6187	0.1691	0.1576	0.1376	0.1435	0.1400	0.1779	0.1748	0.1778	0.1585	0.1514	0.1432	0.1573	0.2458	0.1262	0.1169	0.2136	0.1454	0.1298	0.1251	0.1300	0.1601	0.1687	0.1768	0.1610	0.1996	0.1449	0.1328	0.1653	0.2372	0.1789	0.1678	0.1894	3.0696	0.1803	0.2415	0.2935	2.8604	0.2813	0.2044	0.2590	2.8686	0.2703	0.2112	0.2610	2.8581	0.2041	0.2134	0.2209	2.8560	0.1945	0.2130	0.2711	2.8671	0.2057	0.2464	0.2558	0.2769	0.2686	0.2699	0.3099	4063.17	20.65	21.82	21.33	0.00	0.00	0.00	0.00	
	2/6/2021 5:53 PM	0	0.0606	-0.0005	0.3074	0.5944	0.1709	0.1580	0.1396	0.1428	0.1383	0.1785	0.1751	0.1789	0.1607	0.1527	0.1424	0.1589	0.2446	0.1267	0.1162	0.2120	0.1436	0.1277	0.1228	0.1302	0.1603	0.1692	0.1766	0.1601	0.1984	0.1445	0.1345	0.1632	0.2455	0.1768	0.1695	0.1872	3.0695	0.1825	0.2422	0.2909	2.8602	0.2822	0.2044	0.2565	2.8684	0.2689	0.2104	0.2595	2.8581	0.2009	0.2075	0.2236	2.8560	0.1945	0.2103	0.2696	2.8671	0.2030	0.2444	0.2490	0.2741	0.2686	0.2682	0.3100	4063.17	20.65	2

In [11]:
filename = 'drive/MyDrive/ScanningData_NaturalAging.txt'
with open(filename) as f:
  rows = f.readlines()
  cols = [i for i in range(len(rows[len(rows) - 1].split()))] #this reads the length of the last line and uses that as the number of columns so you don't have to count the number
  data = pd.read_csv(filename, names=cols, sep="\t", header=None, engine='python')
  data[1] = pd.to_datetime(data[1])
  data.set_index(1, inplace=True)
data = data[data.iloc[:, 10:60].gt(0).all(axis=1)]

In [12]:
def funcC(x, a, b, c):
    return a * np.exp(-b * x) + c
def func(x, a, b):
    return a * np.exp(-b * x)
def lin_func(x, a, b):
    return a + b * x
def pol2(x, a, b, c):
  return (a*x**2) + b*x + c

#Ana

In [6]:
mask = (data[68]>20) & (data[68]<22) & (data[2]==1) #a mask to select reasonable temperatures with valid lines
data_ = data[mask]

In [7]:
plt.plot(data_calib_source[68], data_calib_source[13], '.')
plt.ylim(2.2, 2.0)

NameError: ignored

In [ ]:
data_ = data_.resample("D").mean()
data_ = data_.dropna(subset=[11, 12])

In [ ]:
pol1 = poly.polyfit(X, y, 1)
expo = poly.polyfit(X, np.log(y), 1, w=np.sqrt(y))
#ffit = poly.polyval(X, pol1)


In [ ]:
plt.plot(X,y, 'ro', markersize=1.0)
plt.plot(X, poly.polyval(X, pol1))
plt.figure()
plt.plot(X, poly.polyval(X, expo))

In [ ]:
scipy.optimize.curve_fit(lambda t,a,b: a+b*np.log(t),  X,  y, p0=(1, 0.001))

In [ ]:
def temp_graphs(data, temp_target, source, averaging, plotting_just_the_mean):
  """This function takes in the dataframe, the temperature channel you want to use, a 0/1 for the source channel value, and whether you want averaging over a single index/you just want to plot the mean
  It then plots temperature vs current""" 

  tempSensor_dict = {68:[9, 10, 11, 12], 69:[12, 13, 14, 15], 70:[16, 17, 18, 19]} #The temperature channels and their respective channels
  mask = (data[2]==source) & (data[temp_target] > 20) & (data[temp_target] < 22)#pick only those with stuff matching the source, and temperatures that are relatively reasonable
  data = data[mask]

  chanList = tempSensor_dict[temp_target] #This gets the list of 4 channels that you want
  
  chan1 = list(data[chanList[0]]) #The channel data in list form
  norm1 = data[chanList[0]].mean() #The normalization constant for the first channel

  chan2 = list(data[chanList[1]]) #same format as above
  norm2 = data[chanList[1]].mean()
  
  chan3 = list(data[chanList[2]]) #same
  norm3 = data[chanList[2]].mean()

  chan4 = list(data[chanList[3]]) #same
  norm4 = data[chanList[3]].mean()

  temp = list(data[temp_target]) #the temperature data in list form
  tempnorm = data[temp_target].mean() #The normalization constant for the temperature channel

  x = []
  y = []
  fig, ax = plt.subplots(figsize=(12, 4))

  if plotting_just_the_mean: #If you're just plotting the mean for all 4 channel distributions (if the last parameter == true) then it hits this
    mask = (data[2]==source) & (data[68] > 20) & (data[68] < 22) & (data[69] > 20) & (data[69] < 22) & (data[70] > 20) & (data[70] < 22)
    data = data[mask]
    x.append(data[68].mean()) #first temp data
    y.append((data[12].mean() + data[11].mean() + data[10].mean() + data[9].mean())/4) #take the mean of the means

    x.append(data[69].mean()) #seconds temp data
    y.append((data[15].mean() + data[14].mean() + data[13].mean() + data[12].mean())/4) #same

    x.append(data[70].mean()) #third temp data
    y.append((data[19].mean() + data[18].mean() + data[17].mean() + data[16].mean())/4)

    popt, pcov = curve_fit(pol2, x, y) #a 2nd order polynomial worked really well here, so that's what will be used
    print("Fitted Line: y =", popt[2],"* x^2 +",popt[1],"*x +",popt[0]) #print the equation
    ax.scatter(x,y, label = "Average Current Over Channels") #plot the data points
    yerror = math.sqrt(max(statistics.stdev(chan1), statistics.stdev(chan2), statistics.stdev(chan3), statistics.stdev(chan4))) #the error bars are consistent of the largest standard deviation of all of them
    ax.errorbar(x,y,yerr=yerror, ls='none')#put the errorbars down without lines connecting them

    ax.set_xlabel("Temperature")
    ax.set_ylabel("Current")
    title = "Average Temperature VS Average Current for Temperature Sensors"
    if source == 1:
      title += " For Channels with the Source" #Changes the title depending on whether it was with or without the source
    else:
      title += " For Channelse without the Source"
    ax.set_title(title)
    
    func_points = []
    plot_x = np.arange(min(x), max(x) + 0.1, 0.1) #Have the function go over a lot of points, as it makes it more smooth over the range
    for i in plot_x:
      func_points.append(pol2(i,*popt))
    ax.plot(plot_x, func_points, 'r-', label = "Fitted Line")
    
    ax.set_xlim(left=20,right=22)
    plt.grid()
    plt.legend()
    plt.show()

    return #stop the function


  for i in range(len(chan1)): #takes the average of the four for each row
    if temp[i] <= 20 or temp[i] >= 22:
      continue

    if averaging:
      avgcurrent = (chan1[i]/norm1 + chan2[i]/norm2 + chan3[i]/norm3 + chan4[i]/norm4)/4 #average the norms of all the values
      if avgcurrent < 0.8 or avgcurrent > 1.2:
        continue #remove values that are out of these bounds
      x.append(temp[i])
      y.append(avgcurrent)
    else: #This is without averaging over all the entries of a given index
      normList = []
      normList.append(chan1[i]/norm1)
      normList.append(chan2[i]/norm2)
      normList.append(chan3[i]/norm3)
      normList.append(chan4[i]/norm4)
      
      for j in normList:
        if j > 0.8 and j < 1.2: #same bound thing from before
          x.append(temp[i])
          y.append(j)

  popt, pcov = curve_fit(lin_func, x, y)
  print("Fitted Line: y =", popt[1],"* x +",popt[0]) #linear fit for the data (I think a linear fit works better here over a quadratic one)
  
  ax.scatter(x,y, label = "Average Current Over Channels")

  func_points = []
  for i in x:
    func_points.append(lin_func(i, *popt)) #run the fitted function on all the data points

  ax.plot(x, func_points, 'r-', label = "Fitted Line")
  ax.set_xlabel("Temperature")
  ax.set_ylabel("Current")
  chart_title = "Current VS Temperature for Channels " + str(tempSensor_dict[temp_target]).replace("]","").replace("[","")
  if source == 1:
    chart_title += " That Contain the Source"
  else:
    chart_title += " That Does Not Contain the Source"
  ax.set_title(chart_title)

  ax.set_xlim(left=20, right=22)
  plt.grid()
  plt.legend()
  plt.show()


print("-------------------- WITHOUT AVERAGING OVER ALL CHANNELS FOR A GIVEN INDEX --------------------")
print("-------------------- WITH SOURCE --------------------")
temp_graphs(data_calib_source,68,1, False, False)
temp_graphs(data_calib_source,69,1, False, False)
temp_graphs(data_calib_source,70,1, False, False)
print("-------------------- WITHOUT SOURCE --------------------")
temp_graphs(data_calib_source,68,0, False, False)
temp_graphs(data_calib_source,69,0, False, False)
temp_graphs(data_calib_source,70,0, False, False)
print("-------------------- WITH AVERAGING OVER ALL CHANNELS FOR A GIVEN INDEX --------------------")
print("-------------------- WITH SOURCE --------------------")
temp_graphs(data_calib_source,68,1, True, False)
temp_graphs(data_calib_source,69,1, True, False)
temp_graphs(data_calib_source,70,1, True, False)
print("-------------------- WITHOUT SOURCE --------------------")
temp_graphs(data_calib_source, 68,0, True, False)
temp_graphs(data_calib_source, 69,0, True, False)
temp_graphs(data_calib_source, 70,0, True, False)

print("-------------------- CHART WITH JUST AVERAGE DATA --------------------")
temp_graphs(data_calib_source,68,1, True, True)
temp_graphs(data_calib_source, 70,0, True, True)


In [ ]:
def oneChanAtATime(tempSensorChan, source):
  tempSensor_dict = {68:[9, 10, 11, 12], 69:[12, 13, 14, 15], 70:[16, 17, 18, 19]} #The temperature channels and their respective channels
  for i in tempSensor_dict[tempSensorChan]:
    fig, ax = plt.subplots(figsize=(12,4))
    mask = (data[2]==source) & (data[tempSensorChan] > 20) & (data[tempSensorChan] < 22) & (data[i] > 0) #pick only those with stuff matching the source, and temperatures that are relatively reasonable
    data_ = data[mask]
    pedestal = 0
    if source == 1:
      pedestal = data[i][data[2]==0].mean()
      current = (data_[i] - pedestal)
    else:
      current = data_[i]
    current /= current.mean()
    x = data_[tempSensorChan]

    ax.plot(x, current, 'k.', label="SiPM Current")

    popt, pcov = curve_fit(lin_func, x, current)
    print('Chan: '+str(i)+', pedestal: '+str(pedestal))
    print(popt)
    ax.plot(x, lin_func(x, *popt), 'r-', label="Fitted Line")

    ax.set_xlabel("Temperature")
    ax.set_ylabel("Current")
    ax.set_title("Temperature VS Current for Channel " + str(i) + (" Using Only Channels With the Source" if (source == 1) else " Using Only Channels Without the Source"))

    plt.grid()
    plt.show()

oneChanAtATime(68,1)
oneChanAtATime(69,1)
oneChanAtATime(70,1)

In [ ]:
for chan in range(15, 50):

  bad_channels = [19, 39, 43, 47, 48, 49, 50, 51, 55, 56, 57, 58, 59]
  if chan in bad_channels:
      continue

  fig, ax = plt.subplots(figsize=(12, 4))
  mask = (data[chan]>0.2) & (data[2]==1)
  data_ = data[mask]
  pedestal = data[chan][data[2]==0].mean()
  X = (data_.index -  data_.index.to_numpy()[0]).astype('timedelta64[D]')/365
  y = (data_[chan] - pedestal)
  y = y/y[:100].mean()
  y = y*np.exp(X*0.023)
  ax.plot(data_.index, y, 'k.', label="SiPM current")
#  ax.plot(X, y, 'k.', label="SiPM current")

  popt, pcov = curve_fit(func, X, y)
  print('Chan: '+str(chan)+', pedestal: '+str(pedestal))
  print(popt)
  ax.plot(data_.index, func(X, *popt), 'r-', label="Fitted Curve")
#  ax.plot(X, func(X, *popt), 'r-', label="Fitted Curve")
  ax.set_ylim(0.5, 1.1)
  date_form = DateFormatter("%m/%d/%y")
  ax.xaxis.set_major_formatter(date_form)

  plt.grid()
  plt.legend()
  plt.show()

#Dark current vs temp parametrization

In [13]:
   tempSensor_dict = {68:[9, 10, 11, 12], 69:[13, 14, 15], 70:[16, 17, 18, 19]} #The temperature channels and their respective channels

In [14]:
import warnings
import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.rcParams["grid.linestyle"] = (5,15)
plt.rcParams["grid.linewidth"] = 0.5
data_calib_source = data[(data.index>datetime(2021, 2, 1)) & (data.index<datetime(2021, 3, 1))]

In [15]:
signalSlopes = []
DCurrentSlopes = []
chanList = []
chanSlopes = {}
def reset_slopes(): #just a function that resets everything since I'm using global variables
  global signalSlopes, DCurrentSlopes, chanList
  signalSlopes = []
  DCurrentSlopes = []
  chanList = []
  chanSlopes = {}

In [ ]:
def monthlyChannels(date1, date2):
  global signalSlopes, DCurrentSlopes, chanList, chanSlopes
  """takes in the temperature sensor channel, the lower date bound and upper date bound
  dates should be in [yyyy,mm,dd] format"""

  warnings.filterwarnings("ignore")

  if type(date1) != list and type(date2) != list and len(date1) != 3 and len(date2) != 3:
    raise TypeError("Look at the format of your dates!")
  date_limited = data[(data.index>datetime(*date1)) & (data.index<datetime(*date2)) & (data[68] > 20) & (data[68] < 22)]

  if len(date_limited[68]) < 10:
    return
  

  fig, ax = plt.subplots(1, 50, figsize=(12*50,15))
  plt.figtext(0, 0, datetime(*date1).strftime('%b') + " " + str(date1[0]), fontsize=14)

  for i in range(15,65):
    col = i - 15

    ########################################## WITH SOURCE ###############################
    mask = (date_limited[i] > 0) & (date_limited[2] == 1)
    data_ = date_limited[mask]

    pedestal = date_limited[date_limited[2]==0][i].mean()
    x = data_[68]
    y = (data_[i] - pedestal) if not np.isnan(pedestal) else data_[i]
    y = y/y.mean()

    # print(x)
    # print(pedestal)
    # print(y)
    
    chanList.append(i)

    sns.regplot(x=x, y=y, color = 'k', ax=ax[col], fit_reg=None, x_bins = 40, label = "SiPM %d Signal" % i)

    params, pcov = curve_fit(lin_func, x, y)
    signalSlopes.append(params[1])
    try:
      chanSlopes[i].append((datetime(*date1), params[1]))
    except:
      chanSlopes[i] = [(datetime(*date1), params[1])]

    ax[col].plot(x, lin_func(x,*params), 'r-', label = "Linear Fit: " + str(params[1].round(5)) + "x + " + str(params[0].round(5)))

    ax[col].set_title("Temperature VS Signal for Channel " + str(i))
    ax[col].set_xlabel("Temperature")
    ax[col].set_ylabel("Normalized Signal")
    ax[col].set_ylim(0.9, 1.1)
    ax[col].legend()

    
  fig.tight_layout()
    #################################### WITHOUT SOURCE ################################
    # if np.isnan(pedestal):
    #   ax[1].set_title("No data points w/o the source")
    #   DCurrentSlopes.append(None)
    #   continue

    # mask = (date_limited[68] >= 20) & (date_limited[68] <= 22) & (date_limited[i] > 0) & (date_limited[2] == 0)
    # data_ = date_limited[mask]

    # x = data_[68]
    # y = data_[i]
    # y = y/y.mean()

    # sns.regplot(x=x, y=y, color = 'k', ax=ax[1], fit_reg=None, x_bins = 40, label = "SiPM %d Dark Current" % i)

    # params, pcov = curve_fit(lin_func, x, y)
    # DCurrentSlopes.append(params[1])

    # ax[1].plot(x, lin_func(x,*params), 'r-', label = "Linear Fit: " + str(params[1].round(5)) + "x + " + str(params[0].round(5)))

    # ax[1].set_title("Temperature VS Dark Current for Channel " + str(i))
    # ax[1].set_xlabel("Temperature")
    # ax[1].set_ylabel("Normalized Dark Current")
    # ax[1].legend()

# monthlyChannels([2021,2,1], [2021,3,1])
# monthlyChannels([2020,8,1], [2020,9,1])
reset_slopes()

for i in range(3, 12):
  monthlyChannels([2020, i, 1], [2020, i + 1, 1])

monthlyChannels([2020, 12,1], [2021,1,1]) 
#for some reason past a certain point the line no longer draws, I have no clue why

for i in range(1, 3):
  monthlyChannels([2021,i,1], [2021, i + 1, 1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))

plot1x = []
plot1y = []

plot2x = []
plot2y = []

ax[0].set_title("Signal Fit Slope VS Channel")
ax[0].set_xlabel("Channel #")
ax[0].set_ylabel("Signal Fit Slope")

for i in range(len(chanList)):
  if abs(signalSlopes[i]) > 0.1:
    continue4

    
  plot1x.append(chanList[i])
  plot1y.append(signalSlopes[i])

for i in chanSlopes.keys():
  val = chanSlopes[i]
  for j in range(len(val)):
      plot2x.append(val[j][0].strftime('%b %Y'))
      plot2y.append(val[j][1])

  # if DCurrentSlopes[i] == None:
  #   continue
  # ax[0][1].plot(chanList[i], DCurrentSlopes[i], 'bo')

  # ax[1][0].plot(DCurrentSlopes[i], signalSlopes[i], 'bo')

  # if signalSlopes[i] <= 0:
  #   plot4x.append(DCurrentSlopes[i])
  #   plot4y.append(signalSlopes[i])
  

sns.regplot(x=plot1x, y=plot1y, ax=ax[0], x_bins=80, fit_reg = None, label = "Signal Slopes")
params, pcov = curve_fit(lin_func, plot1x, plot1y)
ax[0].plot(plot1x, lin_func(np.array(plot1x), *params), 'r-')

ax[1].scatter(plot2x, plot2y)
# params, pcov = curve_fit(lin_func,plot2x,plot2y)
# ax[1].plot(plot2x, lin_func(np.array(plot2x), *params), 'r-')

# sns.regplot(x=plot4x, y=plot4y, ax=ax[1][1], x_bins=80, fit_reg = None)
# params, pcov = curve_fit(lin_func, plot4x, plot4y)
# ax[1][1].plot(plot4x, lin_func(np.array(plot4x), *params), 'r-')

fig.tight_layout()


In [ ]:
for i in chanSlopes.keys():
  plot2x = []
  plot2y = []
  fit2x = []

  fig, ax = plt.subplots(1, 1, figsize=(12,4))
  ax.set_title("Channel " + str(i) + " Temp Slope VS Time w/o abs(val) > 0.1")
  ax.set_xlabel("Month")
  ax.set_ylabel("Temperature Slope")
  val = chanSlopes[i]
  
  for j in range(len(val)):
    if abs(val[j][1]) > 0.1:
      continue
    if j == 0:
      fit2x.append((val[j][0] - val[0][0]).total_seconds()/86400) #difference in seconds divided by seconds in a day
    else:
      fit2x.append((val[j][0] - val[0][0]).total_seconds()/86400)
    plot2x.append(val[j][0].strftime('%b %Y'))
    plot2y.append(val[j][1])
  ax.scatter(plot2x, plot2y, label = "Data Points")
  # print(fit2x)
  params, pcov = curve_fit(lin_func, fit2x, plot2y)
  ax.plot(plot2x,lin_func(np.array(fit2x), *params), 'r-', label="Fitted line: " + str(round(params[1],7)) + "x + " + str(round(params[0],4)))
  ax.legend()

In [ ]:
for chan in range(15, 65):

  bad_channels = [19, 39, 43, 47, 48, 49, 50, 51, 55, 56, 57, 58, 59]
  if chan in bad_channels:
      continue

# Signal
  fig, ax = plt.subplots(1, 2, figsize=(20, 4))
  pedestal = data_calib_source[chan][data_calib_source[2]==0].mean()
  X = data_calib_source[68][data_calib_source[2]==1]
  y = (data_calib_source[chan][data_calib_source[2]==1] - pedestal)
  y = y/y[:100].mean()
  ax[0].plot(X, y, 'b.', markersize=0.5, label="SiPM %d current" % chan)
  sns.regplot(x=X, y=y, color='k', x_bins=40, fit_reg=None, ax=ax[0])
  popt, pcov = curve_fit(lin_func, X, y)
  ax[0].plot(X, lin_func(X, *popt), 'r-', label="Fitted Curve")
  ax[0].legend()
  ax[0].set_ylim(0.9, 1.1)

# Dark current
  X = data_calib_source[68][data_calib_source[2]==0]
  y = (data_calib_source[chan][data_calib_source[2]==0])
  ax[1].plot(X, y, 'b.', markersize=0.5, label="SiPM %d current" % chan)
  sns.regplot(x=X, y=y, color='k', x_bins=40, fit_reg=None, ax=ax[1])
  popt, pcov = curve_fit(lin_func, X, y)
  ax[1].plot(X, lin_func(X, *popt), 'r-', label="Fitted Curve")